# `S`um of `M`inimums of `C`onvex 

<b>author</b>: @guiguiom 

<b>package name</b>: SMC

<b>package type</b>: [CVXPY](https://www.cvxpy.org) extension

In [1]:
## basic imports 
import numbers
import numpy as np
import cvxpy as cp
from cvxpy.expressions.expression import Expression
from cvxpy.expressions.constants import Constant,Parameter
from scipy.special import softmax
import itertools
%config InlineBackend.figure_format = 'retina'
import warnings
warnings.filterwarnings('ignore')
from IPython.display import clear_output
clear_output()

### utils

In [2]:
# Author: Mathieu Blondel
# License: BSD 3 clause


def proj_simplex_vec(V, z=1):
    n_features = V.shape[1]
    U = np.sort(V, axis=1)[:, ::-1]
    z = np.ones(len(V)) * z
    cssv = np.cumsum(U, axis=1) - z[:, np.newaxis]
    ind = np.arange(n_features) + 1
    cond = U - cssv / ind > 0
    rho = np.count_nonzero(cond, axis=1)
    theta = cssv[np.arange(len(V)), rho - 1] / rho
    return np.maximum(V - theta[:, np.newaxis], 0)

def proj_simplex(v, z=1):
    n_features = v.shape[0]
    u = np.sort(v)[::-1]
    cssv = np.cumsum(u) - z
    ind = np.arange(n_features) + 1
    cond = u - cssv / ind > 0
    rho = ind[cond][-1]
    theta = cssv[cond][-1] / float(rho)
    w = np.maximum(v - theta, 0)
    return w

# from https://github.com/cvxgrp/dccp/blob/master/dccp/linearize.py

def linearize(expr):
    """Returns the tangent approximation to the expression.

    Gives an elementwise lower (upper) bound for convex (concave)
    expressions. No guarantees for non-DCP expressions.

    Args:
        expr: An expression.

    Returns:
        An affine expression.
    """
    if expr.is_affine():
        return expr
    else:
        if expr.value is None:
            raise ValueError(
                "Cannot linearize non-affine expression with missing variable values."
            )
        tangent = np.real(expr.value) #+ np.imag(expr.value)
        grad_map = expr.grad
        for var in expr.variables():
            if grad_map[var] is None:
                return None
            complex_flag = False
            if var.is_complex() or np.any(np.iscomplex(grad_map[var])):
                complex_flag = True
            if var.ndim > 1:
                temp = cp.reshape(
                    cp.vec(var - var.value), (var.shape[0] * var.shape[1], 1)
                )
                if complex_flag:
                    flattened = np.transpose(np.real(grad_map[var])) @ cp.real(temp) + \
                    np.transpose(np.imag(grad_map[var])) @ cp.imag(temp)
                else:
                    flattened = np.transpose(np.real(grad_map[var])) @ temp
                tangent = tangent + cp.reshape(flattened, expr.shape)
            elif var.size > 1:
                if complex_flag:
                    tangent = tangent + np.transpose(np.real(grad_map[var])) @ (cp.real(var) - np.real(var.value)) \
                    + np.transpose(np.imag(grad_map[var])) @ (cp.imag(var) - np.imag(var.value))
                else:
                    tangent = tangent + np.transpose(np.real(grad_map[var])) @ (var - var.value)
            else:
                if complex_flag:
                    tangent = tangent + np.real(grad_map[var]) * (cp.real(var) - np.real(var.value)) \
                    + np.imag(grad_map[var]) * (cp.imag(var) - np.imag(var.value))
                else:
                    tangent = tangent + np.real(grad_map[var]) * (var - var.value) 
        return tangent

### objective creation

In [3]:
### 
### 
### 
# new class
### 
### 
### 

class MinExpr:
    """
    class used to represent the minimum of cvxpy expressions 
    --------------------------------------------------------
    
    Attributes: 
    
    expr_array : list or 2d-array of CONVEX cvxpy Expression 
        LIST
        | every Expression must exhibit same dimension 
        | within each Expression, a single dtype is allowed
    dim : int
        | represents the "depth"/"dimension" of Expression objects stored in expr_array
    NOTES:
        
        ARRAY entry at (pos1,pos2) <=> LIST element at (pos2) regarding dimension pos1
        
    """
    
    def __new__(cls, *args, **kwargs):
        return super().__new__(cls)

    def __init__(self, expr_array):
        self.expressions = None
        if isinstance(expr_array,Expression): # array case
            self.expressions = expr_array.copy()
        elif isinstance(expr_array,list): # list case 
            try:
                self.expressions = cp.vstack(expr_array.copy()).T
                assert isinstance(self.expressions,Expression),'one argument passed does not match cvxpy Expression format'
            except:
                print("error while creating MinExpr object:: check dimensions of list content arguments")
        else:
            print("expr_array should be either a cvxpy Expression array or a list of Expression of equivalent sizes")

        if len(self.expressions.shape)<2:
            self.dim = 1
        elif len(self.expressions.shape)==2:
            self.dim = self.expressions.shape[0]
            if self.dim==1: # flatten() equivalent
                self.expressions = self.expressions[0]
        else: 
            print("expr_array must be a 1 or 2 dimensional")
       
        assert self.expressions.is_convex(), "expr must be convex"

    @property
    def value(self):
        """evaluates min_l=1...n of Expr_l for each pos1"""
        if self.dim==1:
            return np.min(self.expressions.value)
        else: 
            return np.min(self.expressions.value,1)
        
    @property
    def extended_values(self):
        """evaluates every component"""
        return self.expressions.value
    
    @property 
    def ns(self):
        if self.dim==1:
            return [int(np.prod(self.expressions.shape))]
        return self.dim*[int(self.expressions.shape[1])]
    
    def __add__(self, e):
        if isinstance(e, MinExpr):
            return SumMinExpr(list_min_exprs=[self,e])
        elif isinstance(e, SumMinExpr):
            return e+self
        elif isinstance(e, numbers.Number):
            self.expressions += Constant(e)
            return self
        elif isinstance(e, Expression):
            assert e.is_convex(),'e should be a cvxpy CONVEX Expression'
            try:
                self.expressions += e
            except:
                print('dimensions mismatch')
            return self
        else:
            raise ValueError("type %s not supported in __add__" % type(e))
            
    __radd__ = __add__
    
    def __mul__(self, e):
        """multiplies the MinExpr to another object
        
        Parameters
        ----------
        e : numbers.Number or np.ndarray (POSITIVE) or CONSTANT/PARAMETER
        
        Raises
        ------
        ValueError
            if e is not a supported type.
        Returns
        -------
        SumMinExpr
        """
        assert isinstance(e, numbers.Number) or isinstance(e,np.ndarray) or isinstance(e,Constant) or isinstance(e,Parameter),'e should be a scalar or an np.ndarray'
        if ((isinstance(e,Constant) or isinstance(e,Parameter)) and e.is_nonneg()) or (isinstance(e, numbers.Number) and e>=0):
            try:
                self.expressions = cp.multiply(e,self.expressions)
            except:
                raise ValueError('dimension not matching between e and self.expressions')
            return self
        elif np.min(e)>=0:
            try:
                self.expressions = cp.multiply(Constant(e),self.expressions)
            except:
                raise ValueError('dimension not matching between e and self.expressions')
            return self
        else:
            raise ValueError("type %s not supported in __add__" % type(e))
            
    __rmul__ = __mul__ 
    
    def clip(self,lamb):
        """takes the minimum between a MinExpr and a scalar lamb
        
        Parameters
        ----------
        lamb : numbers.Number 

        Returns
        -------
        MinExpr
        """
        assert isinstance(lamb,numbers.Number),'lamb should be a number'
        if self.dim>1:
            self.expressions = cp.hstack((self.expressions,lamb*np.ones((self.dim,1))))
        else:
            self.expressions = cp.hstack((self.expressions,lamb))
            
    def compress(self,weights=None):
        """produces the cvxpy CONVEX Expression sum_l=1...ns weight_l*Expr_l
        
        Parameters
        ----------
        weights : None (default) or np.ndarray
            | weights being positive and of length ns

        Returns
        -------
        Expression 
        """
        loc_ns = self.ns[0]
        if weights is None:
            weights = np.ones(loc_ns)/loc_ns
        if self.dim==1:
            assert len(weights)==loc_ns and (np.array(weights)>=0).all(),'weights should be a positive vector of length ns'
            sw_ = sum(weights)
            return np.array(weights)/sw_@self.expressions
        else:
            assert weights.shape == (self.dim,loc_ns) and (np.array(weights)>=0).all(),'weights should be a positive matrix of size (self.dim,ns)'
            sw_ = np.sum(weights,1)
            return cp.sum(cp.multiply(np.array(weights)/np.outer(np.ones(self.dim),sw_),self.expressions))
        
        
    def param_expand(self):
        """produces the parametric cvxpy CONVEX Expression sum_l=1...ns weight_l*Expr_l
           with newly instanciated cvxpy POSITIVE parameters weight_l for l=1...ns
        
        Returns
        -------
        Expression (parametric)
        """
        new_param = cp.Parameter(self.expressions.shape,nonneg=True)
        return cp.sum(cp.multiply(new_param,self.expressions)),[new_param]
    
### 
### 
### 
# new class
### 
### 
### 

    
class SumMinExpr:
    """
    class used to represent a sum of MinExpr
    ----------------------------------------
    
    Attributes:
    
    main_expr: Expression
        | CONVEX cvxpy Expression that stands outside Min operators (see doc.)
        
    Methods:

    __add__(e)
        -> adds the object to an Expression, MinExpr, SumOfMinExpr, or numbers.Number
    """
    
    def __new__(cls, *args, **kwargs):
        return super().__new__(cls)

    def __init__(self, list_min_exprs,main_fun=None):
        if main_fun is not None:
            assert isinstance(main_fun,Expression),'main (common) expression should be a cvxpy Expression'
            assert main_fun.is_convex(),'main (common) expression should be convex'
            self.main_expr = main_fun.copy()
        else:
            self.main_expr = Constant(0.0)
        self.min_exprs = list_min_exprs
        
    
    @property
    def num_exprs(self):
        'returns N in our formulation of SMC'
        return sum([me.dim for me in self.min_exprs])
    
    @property
    def ns_list(self):
        buf = []
        for me in self.min_exprs:
            buf.append(me.ns)
        return buf

    def __add__(self, e):
        """adds the SumMinExpr to another object
        
        Parameters
        ----------
        e : Expression, MinExpr, SumMinExpr, or numbers.Number
        Raises
        ------
        ValueError
            if e is not a supported type.
        Returns
        -------
        SumMinExpr
        """
        if isinstance(e, MinExpr):
            self.min_exprs += [e]
            return self
        elif isinstance(e, SumMinExpr):
            self.min_exprs += e.min_exprs
            self.main_expr += e.main_expr
            return self
        elif isinstance(e, numbers.Number):
            self.main_expr += Constant(e)
            return self
        elif isinstance(e, Expression):
            assert e.is_convex(),'e should be a cvxpy CONVEX Expression'
            self.main_expr += e
            return self
        else:
            raise ValueError("type %s not supported in __add__" % type(e))
            
    __radd__ = __add__
    
    
    def __mul__(self, e):
        try:
            self.main_expr *= Constant(e)
            for me in self.min_exprs:
                me *= e
        except:
            raise ValueError('dimension not matching between e and either self.main_expr or one of the expr')
        return self
            
    __rmul__ = __mul__

    
    def clip(self, lamb):
        """clips every term of a SumMinExpr to scalar value lamb
        """
        assert isinstance(lamb,numbers.Number),'lamb should be a number'
        for me in self.min_exprs:
            me.clip(lamb)
    
    @property 
    def value(self):
        """overall value"""
        buf = self.main_expr.value
        for me in self.min_exprs:
            buf += np.sum(me.value)
        return buf
    
    @property
    def extended_values(self):
        return [me.extended_values for me in self.min_exprs]
    
    def active(self,relax=1e-5):
        assert relax>=0 and relax<=1,'relax must be in [0,1]'
        ext_val = self.extended_values
        ext_indices = []
        if relax==0:
            for vals in ext_val:
                indices = []
                for row in vals:
                    indices.append(np.argmin(row))
                ext_indices.append(indices)
        else:
            for vals in ext_val:
                indices = []
                minvals = np.min(vals,1)
                maxvals = np.max(vals,1)
                shifted_vals = (np.outer(maxvals,np.ones(vals.shape[1]))-np.array(vals))/(np.outer(maxvals,np.ones(vals.shape[1]))-np.outer(minvals,np.ones(vals.shape[1])))
                for row in shifted_vals:
                    indices.append(list(np.where(row>=1-relax)[0]))
                ext_indices.append(indices)
        return ext_indices
                                                                                    
    def compress(self,weights_list):
        comp = self.main_expr
        assert len(weights_list)==self.num_exprs,'there should be as many weighting factors as MinExpr stored in SumMinExpr object'
        for me,weights in zip(self.min_exprs,weights_list):
            comp += me.compress(weights)
        return comp
    
    def param_expand(self):
        comp = self.main_expr
        new_param_list = []
        for me in self.min_exprs:
            new_term,new_param = me.param_expand()
            comp += new_term
            new_param_list += new_param
        return comp,new_param_list

    
    # TO DO .visualize(variable_values)
    
### 
### 
### 
# static method @todo add Expressions handle with lamb
### 
### 
### 
def minimum(e,lamb):
    assert isinstance(lamb,numbers.Number) or isinstance(lamb,Parameter) or isinstance(lamb,Constant),'lamb should be a number or cp.Parameter/Constant'
    assert isinstance(e, Expression),'e should be a cvxpy Expression'
    if isinstance(lamb,numbers.Number):
        if len(e.shape)<2:
            return SumMinExpr([MinExpr(cp.hstack((cp.reshape(e,(e.shape[0],1)),lamb*np.ones((e.shape[0],1)))))])
        else:
            return SumMinExpr([MinExpr(cp.hstack((e,lamb*np.ones((e.shape[0],1)))))])
    else:
        assert len(lamb.shape)==1 and lamb.shape[0]==1,'lamb should be 1d'
        if len(e.shape)<2:
            return SumMinExpr([MinExpr(cp.hstack((cp.reshape(e,(e.shape[0],1)),lamb*np.ones((e.shape[0],1)))))])
        else:
            return SumMinExpr([MinExpr(cp.hstack((e,lamb*np.ones((e.shape[0],1)))))])

### Problem Creation

In [4]:
class Problem:
    """
    minimizing a SumMinExpr
    --------------------------------------------
    
    Attributes:

    objective : SumMinExpr
    constraints : list
        | list of cvxpy constraints
    vars_ : list
        | cvxpy Variables (pointers) involved in the problem
    custom_param_expand : 
        | @comment
    """

    def __init__(self, objective, constraints=[],custom_param_expand=None,custom_dc_expand=None):
        if isinstance(objective, SumMinExpr):
            self.objective = objective
        elif isinstance(objective,cp.Minimize):
            assert objective.is_dcp(),'objective should be DCP'
            self.objective = SumMinExpr(list_min_exprs=[],main_fun=objective.expr)
        elif isinstance(objective,cp.Maximize):
            assert objective.is_dcp(),'objective should be DCP'
            self.objective = SumMinExpr(list_min_exprs=[],main_fun=-objective.expr)
        else:
            raise ValueError('objective should either be a valid cvxpy DCP Objective or SumMinExpr')
        self.constraints = constraints
        for cstr in self.constraints:
            assert cstr.is_dcp(),'constraints must be CONVEX'
        self.vars_ = []
        for min_expr in self.objective.min_exprs:
            self.vars_ += min_expr.expressions.variables()
        self.vars_ += self.objective.main_expr.variables()
        for constr in self.constraints:
            self.vars_ += constr.variables()
        self.vars_ = list(set(self.vars_))
        if custom_dc_expand is None:
            self.custom_dc_decomp = False
        else:
            self.custom_dc_decomp = True
            self.dc_obj_fun_generator = custom_dc_expand
        if custom_param_expand is None:
            self.custom_set = False
            self.param_obj_fun,self.param_pointers_list = self.objective.param_expand()
        else:
            self.custom_set = True
            self.param_obj_fun,self.param_pointers_list,self.w2p = custom_param_expand[0],custom_param_expand[1],custom_param_expand[2]


    def solve(self, method="max-min", *args, **kwargs):
        """approximately solve the problem 
        
        Parameters
        ----------
        method : str
            | see paper
        args, kwargs
        """
        if method in ['max-min','am','boyd','softmin']:
            return self._solve_RAM(mode=method,*args, **kwargs)
        elif method=='RInDCAe':
            return self._solve_RInDCAe(*args, **kwargs)
        else:
            pass

    def weights_setup(self,mode='equiv'):
        weights = []
        if mode=='equiv':
            for num_s in self.objective.ns_list:
                if len(num_s)==1:
                    weights.append(np.ones(num_s[0])/num_s[0])
                else:
                    weights.append(np.outer(np.ones(len(num_s)),np.ones(num_s[0])/num_s[0]))
        elif mode=='random':
            for num_s in self.objective.ns_list:
                if len(num_s)==1:
                    base_weight = np.random.uniform(0,1,num_s[0])
                    weights.append(base_weight/sum(base_weight))
                else:
                    base_weight = np.random.uniform(0,1,(len(num_s),num_s[0]))
                    weights.append(base_weight/np.outer(np.sum(base_weight,1),np.ones(num_s[0]))) 
        elif mode=='random_int':
            for num_s in self.objective.ns_list:
                if len(num_s)==1:
                    base_weight = np.zeros(num_s[0])
                    base_weight[np.random.choice(nums_s[0])] += 1
                    weights.append(base_weight)
                else:
                    base_weight = np.zeros((len(num_s),num_s[0]))
                    for _ in range(len(num_s)):
                        base_weight[_,np.random.choice(num_s[0])]+=1
                    weights.append(base_weight) 
        return weights
    
    def weights_update(self,mode,weights,coefficient,param=1.0):
        vals_list = self.objective.extended_values
        new_weights = []
        for weight,vals in zip(weights,vals_list):
            if len(vals.shape)<2:
                wcand = np.zeros(vals.shape)
                minid = np.argmin(vals)
                minval = vals[minid]
                w_star = np.zeros(vals.shape)
                w_star[minid] = minval
                ####### CANDIDATE ELECTION #######
                if mode=='am' or coefficient<1e-3:
                    w_hat = w_star
                elif mode=='max-min':
                    maxval = np.max(vals)
                    w_hat = proj_simplex(param*(maxval-vals)/(maxval-minval))
                elif mode=='boyd':
                    gbar = np.array(vals)-minval
                    sgbar = np.sign(gbar)
                    w_hat = weight-param*sgbar
                    w_hat[minid] += param*(1+sgbar[minid])
                    w_hat = proj_simplex(w_hat)
                else: 
                    w_hat = softmax(-param*vals/max(1e-4,np.mean(vals))) # vandessel
                ####### epsilon - SG #######
                num = np.sum((weight-w_star)*vals)
                denum = np.sum((w_hat-w_star)*vals)
                if coefficient*num>=denum:
                    combination = 1
                else:
                    combination = min(1,max(0,coefficient*(num/max(1e-9,denum))))
                wcand = combination*w_hat+(1-combination)*w_star
                combs = [combination]
            elif len(vals.shape)==2:
                wcand = np.zeros(vals.shape)
                minids = np.argmin(vals,1)
                minvals = np.array([vals[_,mid] for _,mid in enumerate(minids)])  
                w_star = np.zeros(vals.shape)
                for _,mid in enumerate(minids):
                    w_star[_,mid] += 1
                ####### CANDIDATE ELECTION #######
                if mode=='am' or coefficient<1e-3:
                    w_hat = w_star
                elif mode=='max-min':
                    maxvals = np.max(vals,1)
                    w_hat = proj_simplex_vec(param*(np.outer(maxvals,np.ones(vals.shape[1]))-np.array(vals))/(np.outer(maxvals,np.ones(vals.shape[1]))-np.outer(minvals,np.ones(vals.shape[1]))))
                elif mode=='boyd':
                    gbar = np.array(vals)-np.outer(minvals,np.ones(vals.shape[1]))
                    sgbar = np.sign(gbar)
                    w_hat = weight-param*np.sign(gbar)
                    for _,mid in enumerate(minids):
                        w_hat[_,mid] += param*(1+sgbar[_,mid])
                    w_hat = proj_simplex_vec(w_hat)
                else: 
                    w_hat = softmax(-param*vals/np.maximum(1e-4,np.abs(np.outer(np.mean(vals,1),np.ones(vals.shape[1])))),axis=1)
                ####### epsilon - SG #######
                combs = []
                for _,w_hat_elem in enumerate(w_hat):
                    num = np.sum((weight[_]-w_star[_])*vals[_])
                    denum = np.sum((w_hat_elem-w_star[_])*vals[_])
                    if coefficient*num>=denum:
                        combination = 1
                    else:
                        combination = min(1,max(0,coefficient*(num/max(1e-9,denum))))
                    wcand[_] = combination*w_hat_elem+(1-combination)*w_star[_]
                    combs.append(combination)
            else:
                print('ValueError: wrong evaluation of component functions... please check dimensions')
            new_weights.append(wcand)
        return new_weights
    
    def _init_variables(self,extra_verb_=False,warm_start=False, warm_start_weights=None,init_weights='equiv', **kwargs):
        #### goal -> create init. x just as it would appear for RAM methods <- ####
        if warm_start_weights is not None:
            weights = warm_start_weights.copy()
        else:
            if isinstance(init_weights,str):
                weights = self.weights_setup(mode=init_weights)
            else:
                try:
                    weights = init_weights(self)
                    # TO DO full assertive statements 
                    assert sum([len(weight) for weight in weights])==self.objective.num_exprs,'user prescribed weights should match objective SumMinExpr.num_exprs'
                    for elem in weights:
                        if len(elem.shape)==1:
                            assert np.min(elem)>=-1e-8 and abs(1-np.sum(elem))<=1e-8,'every weight vector should be POSITIVE and sum up to 1'
                        elif len(elem.shape)==2:
                            assert np.min(elem)>=-1e-8 and np.max(np.abs(1-np.sum(elem,1)))<=1e-8,'every weight vector should be POSITIVE and sum up to 1'
                        else:
                            raise ValueError('tensor like parameters not accepted yet...')
                except:
                    print('init_weights as a callable should take a smc.Problem argument | equiv. weights loaded...')
                    weights = self.weights_setup()
        
        cvx_param_obj_prob = cp.Problem(cp.Minimize(self.param_obj_fun), self.constraints)
        
        if not warm_start:
            try:
                ## param affectation + solve
                if self.custom_set:
                    params = self.w2p(weights)
                    for param,param_pointer in zip(params,self.param_pointers_list):
                        param_pointer.value = param
                else:
                    for weight,param_pointer in zip(weights,self.param_pointers_list):
                        param_pointer.value = np.maximum(0,weight)
            except:
                print('numerical instability for weights/params setting...')
            try:
                cvx_param_obj_prob.solve(**kwargs)
                if extra_verb_:
                    print('-> init ok.')
                return cvx_param_obj_prob
            except:
                print('solver issues... or manual STOP')
                print(cvx_param_obj_prob.value)
                return None

            if cvx_param_obj_prob.status in ['unbounded','infeasible']:
                raise ValueError("weights-fixed problem is %s." % cvx_param_obj_prob.status)
        else:
            print('problem already warm-started...')
            return None
    
    def _solve_RInDCAe(self,maxIters=50,verb_=False,extra_verb_=False,tol=1e-9,total_strong_convex_param=0,
                warm_start=False, warm_start_weights=None,init_weights='equiv',kappa=1.0,TOL_has_moved=1e-3, **kwargs):
        
        """ (Refined Inertial DC Algorithm with extrapolation)
        
        Parameters
        ----------
        @COMPLETE
        **kwargs
            | keyword arguments to be sent to cvxpy solve() function
        
        Returns
        -------
        info : dict
            | dictionary of solver information
        """
        
        assert kappa>=0,'positive kappa required'
        
        _ = self._init_variables(extra_verb_,warm_start,warm_start_weights,init_weights, **kwargs)
        
        var_previous = [var.value for var in self.vars_]
        var_actual = [var.value for var in self.vars_]
        DIVERGENCE_new = 0.0
        
        out_val = []
        
        true_val = np.inf
        
        pre_mu_add = max(0,kappa-total_strong_convex_param/2)
        mu_add = 2*pre_mu_add 
        div_coef = mu_add+total_strong_convex_param-kappa
        
        print('kappa = '+str(kappa))
        print('added mu = '+str(mu_add))
           
        if self.custom_dc_decomp==False:
            f1_obj = self.objective.main_expr
            for min_expr in self.objective.min_exprs:
                f1_obj += cp.sum(min_expr.expressions)
            
        for k in range(maxIters):
            
            DIVERGENCE_previous = DIVERGENCE_new
            
            active_list = self.objective.active(0)
            
            if k>0:
                var_previous = var_actual.copy()
                var_actual = [var.value for var in self.vars_]
            
            if self.custom_dc_decomp==False:

                f2_lin_obj = 0.0

                for active_indices,min_expr in zip(active_list,self.objective.min_exprs):
                    lin_models = linearize(min_expr.expressions)
                    f2_lin_obj = cp.sum(lin_models)
                    for idsel,selected in enumerate(active_indices):
                        f2_lin_obj -= lin_models[idsel,selected]

                fobj = f1_obj-f2_lin_obj
                
            else:
                fobj = self.dc_obj_fun_generator(active_list)
                
            if kappa>0 and k>0: # extrapolation mechanism
                for varid,var in enumerate(self.vars_):
                    fobj -= kappa*cp.sum(cp.multiply(var_actual[varid]-var_previous[varid],var))
                
            if mu_add: # correction for theory safeguard
                for var in self.vars_:
                    fobj += mu_add/2*cp.sum_squares(var)-cp.sum(cp.multiply(mu_add*var.value,var-var.value))
                    
            prob = cp.Problem(cp.Minimize(fobj), self.constraints)
            if extra_verb_:
                print('convexified problem created')
            try:
                prob.solve(**kwargs)
                if extra_verb_:
                    print('convexified problem solved')
            except:
                print('solver issues... or manual STOP')
                    
            DIVERGENCE_new = 0.0
            if div_coef>0:
                for va,vp in zip(var_actual,var_previous):
                    if isinstance(va,numbers.Number):
                        DIVERGENCE_new += div_coef/2*(va-vp)**2
                    else:
                        DIVERGENCE_new += div_coef/2*np.sum((va-vp)**2)
                    
            last_val = true_val
            true_val = self.objective.value
            out_val.append(true_val)

            if verb_:
                print("iter. %04d | Fval. %4.4e " % (k + 1, true_val))
                if extra_verb_ and (k>0 or not warm_start):
                    print('variables: '+str([var.value for var in self.vars_]))
                    
            decr = (last_val+DIVERGENCE_previous)-(true_val+DIVERGENCE_new)
                
            if decr < tol and true_val<out_val[0]-TOL_has_moved:
                if verb_:
                    print ("-> terminated (stopping condition satisfied)")
                break
            else:
                TOL_has_moved = max(-1e-9,TOL_has_moved*2/3-1e-9)
                
        if verb_ and k == maxIters-1:
            print ("-> terminated (maximum number of iterations reached)")
            
        return {'iters':k+1,'stopping_condition':decr,'objective_values':out_val}   
    
    
####
#### (w*a+w_star*(1-a))^T h <= w_star^T h + Ck* (w_last-w_star)^T h
####  a <=  Ck*((w_last-w_star)^T h)/((w-w_star)^T h) 
####
    
    def _solve_RAM(self,mode='am',maxIters=50,verb_=False,extra_verb_=False,tol=1e-9,
                warm_start=False, warm_start_weights=None,init_weights='equiv',kappa=1.0,C_sched=None,param_sched=None,TOL_has_moved=1e-3, **kwargs):
        
        """ (Relaxed Alternating Minimization)
        
        Parameters
        ----------
        maxIters : int
            | maximum number of iterations (default = 50)
        tol : double
            | numerical tolerance for stopping condition (default = 1e-9)
        verb_ : bool
            | whether or not to print information (default = False)
        warm_start : bool
            | whether or not some value affectation has already been conducted on Problem's variables vars_
        warm_start_weights : np.ndarray
            | choice bias; warm start value for a priori weights (default = None)
        init_weights : str or callable
            | 'random','equiv' or homemade init technique
        **kwargs
            | keyword arguments to be sent to cvxpy solve() function
        
        Returns
        -------
        info : dict
            | dictionary of solver information
        """
        if warm_start_weights is not None:
            weights = warm_start_weights.copy()
        else:
            if isinstance(init_weights,str):
                weights = self.weights_setup(mode=init_weights)
            else:
                try:
                    weights = init_weights(self)
                    # TO DO full assertive statements 
                    assert sum([len(weight) for weight in weights])==self.objective.num_exprs,'user prescribed weights should match objective SumMinExpr.num_exprs'
                    for elem in weights:
                        if len(elem.shape)==1:
                            assert np.min(elem)>=-1e-8 and abs(1-np.sum(elem))<=1e-8,'every weight vector should be POSITIVE and sum up to 1'
                        elif len(elem.shape)==2:
                            assert np.min(elem)>=-1e-8 and np.max(np.abs(1-np.sum(elem,1)))<=1e-8,'every weight vector should be POSITIVE and sum up to 1'
                        else:
                            raise ValueError('tensor like parameters not accepted yet...')
                except:
                    print('init_weights as a callable should take a smc.Problem argument | equiv. weights loaded...')
                    weights = self.weights_setup()
        
        cvx_param_obj_prob = cp.Problem(cp.Minimize(self.param_obj_fun), self.constraints)
        
        out_val = []
        
        last_val = np.inf
        decr = np.inf
        
        if C_sched is None:
            C = lambda k: 2/(k**(1/2)+3) 
        else:
            C = lambda k: C_sched(k)
            
        if param_sched is None:
            kap = lambda k: kappa
        else:
            kap = lambda k: param_sched(k)
            
        for k in range(maxIters):
            # x step, skipped if warm_start=True and k=0
            if not warm_start or k > 0:
                try:
                    ## param affectation + solve
                    if self.custom_set:
                        params = self.w2p(weights)
                        for param,param_pointer in zip(params,self.param_pointers_list):
                            param_pointer.value = param
                    else:
                        for weight,param_pointer in zip(weights,self.param_pointers_list):
                            param_pointer.value = np.maximum(0,weight)
                except:
                    print('numerical instability for weights/params setting...')
                    return {'iters':k,'stopping_condition':decr,'objective_values':out_val}
                try:
                    cvx_param_obj_prob.solve(**kwargs)
                except:
                    print('solver issues... or manual STOP')
                    print(cvx_param_obj_prob.value)
                prob_value = cvx_param_obj_prob.value
                decr = max(0.0,last_val-prob_value)
                if cvx_param_obj_prob.status in ['unbounded','infeasible']:
                    raise ValueError("weights-fixed problem is %s." % cvx_param_obj_prob.status)
            else:
                prob_value = self.objective.value
                    
            last_val = prob_value
            true_val = self.objective.value
            out_val.append(true_val)

            if verb_:
                print("iter. %04d | Fval. %4.4e | BICval. %4.4e" % (k + 1, true_val,prob_value))
                if extra_verb_ and (k>0 or not warm_start):
                    print('used weights: '+str(weights))
                    if self.custom_set:
                        print('used params: '+str(params))
                    print('variables: '+str([var.value for var in self.vars_]))
                    
            # weights step
            if warm_start:
                new_weights = self.weights_update(mode,weights,0,kap(k)) # forced full alternating-minimization 
            else:
                new_weights = self.weights_update(mode,weights,C(k),kap(k))
                
            if decr < tol and true_val<out_val[0]-TOL_has_moved:
                if verb_:
                    print ("-> terminated (stopping condition satisfied)")
                break
            else:
                TOL_has_moved = max(-1e-9,TOL_has_moved*2/3-1e-9)
                weights = new_weights.copy()
                
        if verb_ and k == maxIters-1:
            print ("-> terminated (maximum number of iterations reached)")
            
        return {'iters':k+1,'stopping_condition':decr,'objective_values':out_val}       
    
    @property 
    def value(self):
        for cstr in self.constraints:
            if cstr.value()==False:
                return np.inf # infeasibility
        return self.objective.value